In [1]:
%pip install rasterio

  Using cached rasterio-1.3.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.3 MB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached affine-2.4.0-py3-none-any.whl (15 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd 
import os
import requests
import matplotlib.pyplot as plt 
import rasterio


In [2]:
lat = 49.083333
lon = -122.350
# lat: 49.083333    lon: -122.350
url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&accept-language=en'
res = requests.get(url)



In [3]:
res.json()['address']['country']

'Canada'

In [3]:
income_US = pd.read_csv("./Energy_graph/data/metadata/kaggle_income.csv", encoding="latin-1")

income_US
def find_closest(df, lat, lon):
    df['distance'] = df.apply(lambda row: geodesic((lat, lon), (row['Lat'], row['Lon'])).miles, axis=1)
    closest_row = df.loc[df['distance'].idxmin()]
    return closest_row

# GDP PPP in US dollars(2017)

In [4]:
gpd_PPP =  pd.read_csv("./Energy_graph/data/metadata/gdp-per-capita-worldbank.csv")
df = gpd_PPP[gpd_PPP["Year"] == 2020].drop(columns=["Year", "Code"])

df.reset_index(inplace=True, drop=True)
df.set_index("Entity", inplace=True)

In [5]:
df.loc[res.json()['address']['country']].values[0]

46064.254

# Average wages

In [12]:
path = "./Energy_graph/data/metadata/Wages/DP_LIVE_25072023100757533.csv"

wages = pd.read_csv(path)
# average wages in 2021 in the OECD countries
wages_processed = wages[wages["TIME"] == 2021].drop(columns=["TIME", "INDICATOR", "SUBJECT", "MEASURE", "FREQUENCY", "Flag Codes"]).set_index("LOCATION")
wages_processed

,Value
LOCATION,
AUS,60681.498547
AUT,65402.316297
BEL,65520.821305
CAN,59568.782789
CZE,36269.707992
DNK,66503.047789
FIN,52785.319629
FRA,52542.188916
DEU,60291.241771


# Population density


In [25]:
# from https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11

path = "./Energy_graph/data/metadata/test/Density_5min/gpw_v4_population_density_rev11_2020_2pt5_min.asc"
# os.listdir(path)

In [26]:

latitude =49.083333
longitude = -122.35
from rasterio.warp import transform

# Open the dataset
with rasterio.open(path) as ds:
    # Get the geographic bounds of the dataset
    left, bottom, right, top = ds.bounds

    # Coordinate transformation function
    transform_fn = rasterio.transform.from_origin(left, top, ds.res[0], ds.res[1])

    # Convert your latitude and longitude to dataset's coordinate system
    x, y = transform('EPSG:4326', ds.crs, [longitude], [latitude])

    # Fetch the population density value at the given coordinates
    row, col = ds.index(x[0], y[0])
    population_density = ds.read(1)[row, col]

print(population_density)

2496.01


## Elevation and Topography


In [7]:
# TODO get elevation data
lat = 52.52
lon = 13.41
elevation = requests.get(f"https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}")
elevation.json()["elevation"][0]

38.0

# Education


In [31]:
# TODO Get education level data
# https://data.oecd.org/eduatt/population-with-tertiary-education.htm
path = "./Energy_graph/data/metadata/Education/adult_education_levels.csv"

df = pd.read_csv(path)
df[df["TIME"] == 2020].drop(columns=["TIME", "Flag Codes", "MEASURE", "FREQUENCY", "INDICATOR"]).set_index("LOCATION")

,SUBJECT,Value
LOCATION,,
AUS,BUPPSRY,16.244976
AUS,TRY,49.337452
AUS,UPPSRY,34.417572
AUT,BUPPSRY,14.341329
AUT,TRY,34.205894
...,...,...
CRI,TRY,25.112001
CRI,UPPSRY,17.556700
G20,BUPPSRY,34.704054


# Energy sources


In [13]:
# https://data.oecd.org/

path = "./Energy_graph/data/metadata/Energy/DP_LIVE_25072023101159292.csv"
df = pd.read_csv(path)
df

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1960,31.483,NaN
1,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1961,32.664,NaN
2,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1962,34.085,NaN
3,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1963,35.961,NaN
4,AUS,PRYENRGSUPPLY,TOT,MLN_TOE,A,1964,37.404,NaN
...,...,...,...,...,...,...,...,...
18471,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2017,0.175,NaN
18472,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2018,0.176,NaN
18473,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2019,0.178,NaN
18474,UGA,PRYENRGSUPPLY,TOT,TOE_1000USD,A,2020,0.172,NaN


# Walk score US&CANADA only


In [ ]:
# https://www.walkscore.com/professional/api.php

# Broadband access
https://data.oecd.org/broadband/households-with-broadband-access.htm

In [14]:
path = "./Energy_graph/data/metadata/DP_LIVE_25072023101945075.csv"
df = pd.read_csv(path)
df

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,SWE,HHBROADHBAND,2INCOMEQ,PC,A,2006,48.836136,NaN
1,SWE,HHBROADHBAND,2INCOMEQ,PC,A,2007,66.759302,NaN
2,SWE,HHBROADHBAND,2INCOMEQ,PC,A,2008,67.526682,NaN
3,SWE,HHBROADHBAND,2INCOMEQ,PC,A,2009,79.523932,NaN
4,SWE,HHBROADHBAND,2INCOMEQ,PC,A,2010,85.833308,NaN
...,...,...,...,...,...,...,...,...
606,POL,HHBROADHBAND,2INCOMEQ,PC,A,2006,16.302662,NaN
607,POL,HHBROADHBAND,2INCOMEQ,PC,A,2007,17.416610,NaN
608,POL,HHBROADHBAND,2INCOMEQ,PC,A,2008,26.691832,NaN
609,POL,HHBROADHBAND,2INCOMEQ,PC,A,2009,41.874895,NaN


# Energy prices EEA
Gas:
https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_202__custom_7003504/default/table?lang=en

Electricity:
https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_204__custom_7006505/default/table?lang=en

In [36]:
# gas prices in PPS in kWh for medium sized households 

path ="./Energy_graph/data/metadata/Energy/Gas_prices_EEA.csv"
df = pd.read_csv(path)
df = df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "product", "nrg_cons", "unit", "tax", "currency", "OBS_FLAG"])
df

# df[df["nrg_cons"] != "GJ20-199"]

,geo,TIME_PERIOD,OBS_VALUE
0,AL,2021-S1,0.0000
1,AL,2021-S2,0.0000
2,AL,2022-S1,0.0000
3,AT,2007-S2,0.0571
4,AT,2008-S1,0.0537
...,...,...,...
893,UK,2018-S1,0.0406
894,UK,2018-S2,0.0457
895,UK,2019-S1,0.0423
896,UK,2019-S2,0.0436


In [37]:
# electricty prices in PPS per kWh for diffrent yearly consumption levels
path = "./Energy_graph/data/metadata/Energy/Electricity_prices_EEA.csv"
df = pd.read_csv(path)
# df[(~df["nrg_cons"].isin(['KWH_LT1000', 'TOT_KWH']))].drop(columns=["LAST UPDATE", "freq", "product","OBS_FLAG", "DATAFLOW", "currency", "tax", "unit"])

df.drop(columns=["LAST UPDATE", "freq", "product","OBS_FLAG", "DATAFLOW", "currency", "tax", "unit"])
# df

,nrg_cons,geo,TIME_PERIOD,OBS_VALUE
0,KWH1000-2499,AL,2011-S1,0.2632
1,KWH1000-2499,AL,2011-S2,0.2632
2,KWH1000-2499,AL,2012-S1,0.2586
3,KWH1000-2499,AL,2012-S2,0.2586
4,KWH1000-2499,AL,2013-S1,0.2578
...,...,...,...,...
3311,KWH5000-14999,UK,2018-S1,0.1491
3312,KWH5000-14999,UK,2018-S2,0.1629
3313,KWH5000-14999,UK,2019-S1,0.1645
3314,KWH5000-14999,UK,2019-S2,0.1748


In [38]:
# average electricity prices in PPS per kWh for diffrent yearly consumption levels
df_grouped = df.groupby(['geo', 'TIME_PERIOD'])['OBS_VALUE'].mean().reset_index()
df_grouped

,geo,TIME_PERIOD,OBS_VALUE
0,AL,2011-S1,0.263200
1,AL,2011-S2,0.263200
2,AL,2012-S1,0.258600
3,AL,2012-S2,0.258600
4,AL,2013-S1,0.257800
...,...,...,...
1103,UK,2018-S1,0.166267
1104,UK,2018-S2,0.180733
1105,UK,2019-S1,0.183633
1106,UK,2019-S2,0.193033


# Cooling and heating degree days by country

https://ec.europa.eu/eurostat/databrowser/view/NRG_CHDD_A/default/table?lang=en

explanation:https://ec.europa.eu/eurostat/cache/metadata/en/nrg_chdd_esms.htm

In [48]:
#  regional data https://ec.europa.eu/eurostat/databrowser/view/NRG_CHDDR2_A/default/table?lang=en

path = "./Energy_graph/data/metadata/Energy/Heating_cooling_index.csv"
df = pd.read_csv(path)

# drop useless columns
df.drop(columns=["DATAFLOW", "LAST UPDATE","unit", "freq", "OBS_FLAG"], inplace=True)


# Pivot the DataFrame to reshape it. 
# The unique values in column 'indic_nrg' become the new columns in the output DataFrame.
df_pivot = df.pivot(index=['geo', 'TIME_PERIOD'], columns='indic_nrg', values='OBS_VALUE')

In [49]:
df_pivot

indic_nrg          CDD      HDD
geo TIME_PERIOD                
AT  2007         25.74  3363.28
    2008          7.30  3444.43
    2009          9.94  3504.84
    2010         25.27  3905.88
    2011         17.63  3409.28
...                ...      ...
SK  2018         53.06  2923.83
    2019         65.09  2899.05
    2020         19.95  3046.66
    2021         62.28  3382.63
    2022         67.77  3042.68

[464 rows x 2 columns]

In [59]:
%pip install --upgrade holidays

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.7/695.7 kB 14.1 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


# Public holidays


In [60]:


# https://pypi.org/project/holidays/

from datetime import date
import holidays

us_holidays = holidays.country_holidays('US')

date(2021, 1, 1) in us_holidays

True

In [63]:
list(us_holidays.keys())

[datetime.date(2021, 1, 18),
 datetime.date(2021, 2, 15),
 datetime.date(2021, 10, 11),
 datetime.date(2021, 1, 1),
 datetime.date(2021, 12, 31),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 6, 19),
 datetime.date(2021, 6, 18),
 datetime.date(2021, 7, 4),
 datetime.date(2021, 7, 5),
 datetime.date(2021, 9, 6),
 datetime.date(2021, 11, 11),
 datetime.date(2021, 11, 25),
 datetime.date(2021, 12, 25),
 datetime.date(2021, 12, 24)]